### Instalação dos pacotes necessários

Pandas: Biblioteca para manipulação e análise de dados
PyArrow: Biblioteca para leitura e escrita de arquivos parquet
Spotipy: Biblioteca para acesso a API do Spotify

Use o comando abaixo para instalar os pacotes necessários:
```bash
$ pip install pandas pyarrow spotipy
```

Ou instale as dependências diretamente do arquivo `requirements.txt`:
```bash
$ pip install -r requirements.txt
```

### Configuração da API do Spotify com um .env

Para acessar a API do Spotify é necessário criar um aplicativo no [Spotify for Developers](https://developer.spotify.com/dashboard/applications) e obter as credenciais de acesso.

Crie um arquivo `.env` na raiz do projeto e adicione as seguintes variáveis de ambiente:
```bash
SPOTIPY_CLIENT_ID=seu_client_id
SPOTIPY_CLIENT_SECRET=sua_client_secret
```

In [45]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import glob

In [46]:
env_vars = {}

# Le o arquivo .env
with open('.env') as f:
    env_vars = dict(
        tuple(line.replace('"', '').replace("'", '').strip().split('=', 1)) for line in f
    )

# Cria o objeto de autenticação
client_credentials_manager = SpotifyClientCredentials(
    client_id=env_vars['SPOTIPY_CLIENT_ID'],
    client_secret=env_vars['SPOTIPY_CLIENT_SECRET']
)

In [47]:
# Configurando as credenciais do cliente
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Função para obter as 100 playlists mais populares de música independente do Brasil
def get_playlists(query = '', n = 1, offset = 50, market = 'BR'):
    all_tracks = 0
    all_playlists = []
    for i in range(n):
        try:
            playlists = sp.search(query, limit=offset, offset=i*offset, type='playlist', market = market)
            # Printando o nome das playlists
            for item in playlists['playlists']['items']:
                play_tracks = 0
                print(f"Playlist encontrada: {item['name']}")
                all_playlists.append(item)
                # Obtendo as tracks da playlist
                play_id = item['id']
                print(f"Obtendo músicas da playlist: {play_id}")
                tracks = get_playlist_tracks(play_id, market = market)
                # Salvando o objeto de tracks em um arquivo parquet
                df = pd.DataFrame(tracks)
                # Convertendo o DataFrame para um Arrow Table
                table = pa.Table.from_pandas(df)
                # Salvando o arquivo parquet
                pq.write_table(table, f'tracks_{play_id}.parquet')
                # Incrementando o contador de músicas
                play_tracks += len(tracks)
                print(f"Músicas encontradas: {play_tracks}")
                print("-------------------------------------------------------")
                all_tracks += play_tracks
        except Exception as e:
            print("Erro ao obter playlists: ", e)
            continue

    # Salvando o objeto de playlists em um arquivo parquet
    df = pd.DataFrame(all_playlists)
    # Convertendo o DataFrame para um Arrow Table
    table = pa.Table.from_pandas(df)
    # Salvando o arquivo parquet
    pq.write_table(table, f'playlists_{query}.parquet')
    print(f"Playlists encontradas: {len(all_playlists)}")
    print(f"Musicas encontradas: {all_tracks}")
    return all_playlists

# Função para obter todas as músicas de uma playlist
def get_playlist_tracks(playlist_id, limit = 100, market = 'BR'):
    all_tracks = []
    try:
        results = sp.playlist_tracks(playlist_id, limit = limit, market = market)
        while results['next']:
            results = sp.next(results)
            all_tracks.extend(results['items'])
    except Exception as e:
        print("Erro ao obter músicas da playlist: ", e)
    return all_tracks

# Obtendo as n*offset playlists retornadas pela função get_playlists
top_playlists = get_playlists(query = 'underground carioca', n = 1, offset = 5, market = 'BR')


Playlist encontrada: CEP 20.000
Obtendo músicas da playlist: 6JmtTvFjLMqwuw66DjdmdC
Músicas encontradas: 0
-------------------------------------------------------
Playlist encontrada: Underground 90 e 00
Obtendo músicas da playlist: 0ssNBFYEvCHITWiE5GC4EG
Músicas encontradas: 0
-------------------------------------------------------
Playlist encontrada: UNDERGROUND CARIOCA 🌐📈
Obtendo músicas da playlist: 2vXSVAsdyzPSiCiPb5EUxt
Músicas encontradas: 0
-------------------------------------------------------
Playlist encontrada: Underground Anos 90/2000
Obtendo músicas da playlist: 17zzqYemdM2H5YVkuYRprQ
Músicas encontradas: 3
-------------------------------------------------------
Playlist encontrada: ELAM ANOS 90/2000
Obtendo músicas da playlist: 33y3y1dssJsmq7TwAT4fd5
Músicas encontradas: 0
-------------------------------------------------------
Playlists encontradas: 5
Musicas encontradas: 3


In [48]:
# Unificar todos os arquivos parquet de tracks do diretório

# Lista todos os arquivos parquet do diretório
files = glob.glob('tracks_*.parquet')

for file in files:
    table = pq.read_table(file)
    df = table.to_pandas()

    new_df = pd.DataFrame()

    for row in df.iterrows():
        if not row[1]['track']:
            continue
        # Converte cada linha em uma entrada na tabela com colunas
        new_row = {
            'id': row[1]['track']['id'] if row[1]['track']['id'] else None,
            'nome': row[1]['track']['name'] if row[1]['track']['name'] else None,
            'popularidade': row[1]['track']['popularity'] if row[1]['track']['popularity'] else None,
            'track': row[1]['track']['track'] if row[1]['track']['track'] else None,
            'num_track': row[1]['track']['track_number'] if row[1]['track']['track_number'] else None,
            'tipo': row[1]['track']['type'] if row[1]['track']['type'] else None,
            'album': row[1]['track']['album']['name'] if row[1]['track']['album']['name'] else None,
            'artistas': [{'nome': artist['name'], 'id': artist['id']}
                if artist and artist['name'] and artist['id'] else None
                for artist in row[1]['track']['artists']],
            'num_disc': row[1]['track']['disc_number'] if row[1]['track']['disc_number'] else None,
            'duracao': row[1]['track']['duration_ms'] if row[1]['track']['duration_ms'] else None,
            'explicita': row[1]['track']['explicit'] if row[1]['track']['explicit'] else None,
            'ids_externos': row[1]['track']['external_ids'] if row[1]['track']['external_ids'] else None,
            'urls_externas': row[1]['track']['external_urls'] if row[1]['track']['external_urls'] else None,
            'href': row[1]['track']['href'] if row[1]['track']['href'] else None,
            'local': row[1]['track']['is_local'] if row[1]['track']['is_local'] else None,
            'tocavel': row[1]['track']['is_playable'] if row[1]['track']['is_playable'] else None,
            'preview_url': row[1]['track']['preview_url'] if row[1]['track']['preview_url'] else None,
            'uri': row[1]['track']['uri'] if row[1]['track']['uri'] else None,
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

    # Exportando o DataFrame para um novo arquivo parquet
    table = pa.Table.from_pandas(new_df)
    pq.write_table(table, 'pre_proc_' + file)

/tmp/ipykernel_7312/631843341.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
/tmp/ipykernel_7312/631843341.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
/tmp/ipykernel_7312/631843341.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude emp

In [49]:
files = glob.glob('playlists_*.parquet')

for file in files:
    table = pq.read_table(f'playlists_artistas independentes brasil 2023.parquet')
    df = table.to_pandas()

    new_df = pd.DataFrame()

    for row in df.iterrows():
        # Converte cada linha em uma entrada na tabela com colunas
        # Nome da Playlist, Descrição, Link Externo, Proprietário, Total de Faixas
        new_row = {
            'id': row[1]['id'],
            'nome': row[1]['name'],
            'descricao': row[1]['description'],
            'link': row[1]['external_urls']['spotify'],
            'proprietario': row[1]['owner']['display_name'],
            'n_faixas': row[1]['tracks']['total']
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

    # Exportando o DataFrame para um novo arquivo parquet
    table = pa.Table.from_pandas(new_df)
    pq.write_table(table, 'pre_proc_' + file)